## Data Engineering Capstone Project

### Project Summary
The project explores US immigration data alongside US city demographics and world temperature data. The data is explored, then aggregated to build an ETL pipeline inside the notebook. The notebook format was chosen to allow intituative investigation of the data, while still building up the ETL model.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, date_add, col
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd

## Step 1: Scope the Project and Gather Data

### Scope 
This project will use data from multiple sources and create both fact and dimension tables to allow the user to query for links between destination of travel and the temperature of the travel destination. For example we can query the numbers arriving in a city and its temperature on different months, the length of the stay and the reason.

### Describe and Gather Data 
The following datasets are used in the project:

- I94 Immigration Data: This data comes from the US National Tourism and Trade Office and includes the contents of the i94 form on entry to the united states. A data dictionary is included in the workspace. It is provided in SAS7BDAT format which is a binary database storage format.
- World Temperature Data: This dataset comes from Kaggle and includes the temperatures of various cities in the world fomr 1743 to 2013. The dataset can be found [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

Now lets take a look at the data

#### Sample Immigration Data

The immigration dataset is large, containing around 3 million rows. A sample file can be used to perform an initial exploration.

In [2]:
df_immigration_sample = pd.read_csv('immigration_data_sample.csv')

Lets take a look at the columns:

In [3]:
df_immigration_sample.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

The columes are defined in the data dictionary included with the workspace (I94_SAS_Labels_Description.SAS). A sample of these are:

- i94yr: 4 digit year
- i94mon: Numeric month
- i94cit: 3 digit code of origin city
- i94port: 3 character code of destination USA city
- arrdate: Arrival date in the USA
- i94mode: 1 digit travel code
- depdate: Departure date from the USA
- i94visa: Reason for immigration

Let's increase the number of columns that can be displayed at once to have a better look at the data:

In [4]:
pd.set_option('display.max_columns', 50)
df_immigration_sample.head(5)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


#### Temperature Data
Tourism often displays a strong connection to climate data, so this world temperature data will help explore that. Lets load the data and take a look at its shape and row count.

In [5]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(fname)
df_temperature.shape

(8599212, 7)

In [6]:
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [7]:
df_temperature.shape[0]

8599212

#### Immigration Data Frame
We are going to load a full month os immigration data to work with. We also setup a spark session at this point, and will use this in the rest of the notebook.

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.10")\
    .enableHiveSupport().getOrCreate()

def sas_file_to_spark_df(file):
    return spark.read.format('com.github.saurfang.sas.spark').load(file)

df_immigration = sas_file_to_spark_df('/data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

Lets check the data looks like the sample.

In [9]:
df_immigration.count()

3096313

In [10]:
df_immigration.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

#### Valid Ports
Create a dictionary of valid ports from the data dictionary as an additional data source.

In [11]:
### i94_port_lines are 303 - 962

import re
from pprint import pprint

with open('I94_SAS_Labels_Descriptions.SAS') as f:
    lines = f.readlines()   

exp = re.compile(r'\'(.*)\'.*\'(.*)\'')
valid_ports = {}
for line in lines[302:961]:
    match = exp.search(line)
    valid_ports[match.group(1)]=[match.group(2)]

#pprint(valid_ports)

## Step 2: Explore and Assess the Data
### Explore and Clean Data 
Identify data quality issues, like missing values, duplicate data, etc. During the process we will build up some of the functions we will use in the ETL.

#### Immigration Data
For the immigration data, we want to remove all rows where the destination city code i94port is not a valid value. We can find invalid codes in the data dictionary (I94_SAS_Labels_Description.SAS).

In [12]:
df_immigration.columns

['cicid',
 'i94yr',
 'i94mon',
 'i94cit',
 'i94res',
 'i94port',
 'arrdate',
 'i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'admnum',
 'fltno',
 'visatype']

There is a lot of data here. For the purpose of our pipeline, and since we are interested in looking at correlations between destination and temperature only, we keep the following columns: i94yr, i94mon, i94cit, i94port, i94mode, i94bir, arrdate, depdate and i94visa

In [13]:
def clean_immigration_data(immigration_data):
    
    # read immigration data into Spark
    df_immigration = sas_file_to_spark_df(immigration_data)

    # filter out entries where i94port is invalid
    df_immigration = df_immigration.filter(df_immigration.i94port.isin(list(valid_ports.keys())))

    # filter out the columns we want to keep
    columns_to_keep = {'i94yr', 'i94mon', 'i94cit', 'i94port', 'i94mode', 'i94bir', 'arrdate', 'depdate', 'i94visa'}
    df_immigration = df_immigration.filter(df_immigration.i94port.isin(list(valid_ports.keys())))
    df_immigration = df_immigration.select([c for c in df_immigration.columns if c in columns_to_keep])
    
    # return the dataframe
    return df_immigration

In [14]:
# test the function
immigration_test_file = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration_test = clean_immigration_data(immigration_test_file)
df_immigration_test.show()

+------+------+------+-------+-------+-------+-------+------+-------+
| i94yr|i94mon|i94cit|i94port|arrdate|i94mode|depdate|i94bir|i94visa|
+------+------+------+-------+-------+-------+-------+------+-------+
|2016.0|   4.0| 692.0|    XXX|20573.0|   null|   null|  37.0|    2.0|
|2016.0|   4.0| 254.0|    ATL|20551.0|    1.0|   null|  25.0|    3.0|
|2016.0|   4.0| 101.0|    WAS|20545.0|    1.0|20691.0|  55.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20567.0|  28.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20567.0|   4.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20555.0|  57.0|    1.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20558.0|  63.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20558.0|  57.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20553.0|  46.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20562.0|  48.0|    1.0|
|2016.0|   4.0| 101.0|    NYC|20545.0|    1.0|20671.0|  52.0|    2.0|
|2016.0|   4.0| 101.

#### Temperature Data

Lets look at the temperature data. 

In [15]:
unique_temps = df_temperature.AverageTemperature.unique()
print(max(unique_temps))
print(min(unique_temps))

39.651
-42.704


Average temperature values look ok. We will clean as follows:

- We want to drop all entries where AverageTemperature is NaN.
- We will drop all entries with duplicate locations
- Add the i94port of the location in each entry so it can be joined in the data model.

In [16]:
def csv_file_to_spark_df(file):
    return spark.read.format('csv').option('header', 'true').load(file)

# define a quick function to convert the city to port value
@udf()
def convert_city_to_port(city):
    for key in valid_ports:
        if city.lower() in valid_ports[key][0].lower():
            return key

def clean_temperature_data(temperature_data_file):
    df_temperature = csv_file_to_spark_df(temperature_data_file)
    
    # remove NaN entries
    df_temperature = df_temperature.filter(df_temperature.AverageTemperature != 'NaN')
    
    # drop duplicates
    df_temperature = df_temperature.dropDuplicates(['City', 'Country'])
    
    # add port code
    df_temperature = df_temperature.withColumn('i94port', convert_city_to_port(df_temperature.City))
    
    # return a data frame that filters any null port codes
    return df_temperature.filter(df_temperature.i94port != 'null')

df_temperature = clean_temperature_data('../../data2/GlobalLandTemperaturesByCity.csv')

In [17]:
df_temperature.show()

+----------+-------------------+-----------------------------+---------+--------------------+--------+---------+-------+
|        dt| AverageTemperature|AverageTemperatureUncertainty|     City|             Country|Latitude|Longitude|i94port|
+----------+-------------------+-----------------------------+---------+--------------------+--------+---------+-------+
|1856-01-01|             26.901|                        1.359|      Ife|             Nigeria|   7.23N|    4.05E|    888|
|1852-07-01|             15.488|                        1.395|    Perth|           Australia|  31.35S|  114.97E|    PER|
|1828-01-01|             -1.977|                        2.551|  Seattle|       United States|  47.42N|  121.97W|    SEA|
|1743-11-01|              2.767|                        1.905| Hamilton|              Canada|  42.59N|   80.73W|    HAM|
|1849-01-01|  7.399999999999999|                        2.699|  Ontario|       United States|  34.56N|  116.76W|    ONT|
|1821-11-01|              2.322|

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

The model is quite simple, being made up of two staging tables and a final table. Using the staging tables gives us a chance to clean and manipulate the data before it is joined into the final table. The final table will be used to run queries against and provide details on each immigration event. For example, we can run simple queries against this table about the numbers of visitors to each city in a given month and both the temperature and reason for for the visit.

These are detailed as follows:

Staging table staging_demographics holds the cleaned data from the immigration dataset. Each row represents an act of immigration into the US. It will contain the following columns from the immigration data:

- I94Year: Year as a 4 digit number
- I94Month: Month as a numeric
- I94City: 3 digit code of origin city
- I94Port: 3 character code of destination city
- I94Mode: 1 digit travel code
- I94Birthday: Age of Respondent in Years
- ArrDate: Arrival date
- DepDate: Departure date
- I94Visa: Reason for immigration (Business/Pleasure/Student)

The second staging table, staging_temperate, will contain the cleaned city temperature data from the temperature dataset. Each row represents a temperature measurement. It will contain:

- I94Port: Code of destination city (pulled from immigration data during cleanup step)
- AverageTemperature: Average temperature
- City: City name
- Country: Country name
- Latitude: Latitude
- Longitude: Longitude

The final table, final_immigration, its made from joining the two staging tables:

- Year: Year as a 4 digit number
- Month: Month as a numeric
- City: 3 digit code of origin city
- Destination: 3 character code of destination city
- Mode: 1 digit travel code
- Birthday: Age of Respondent in Years
- ArrDate: Arrival date
- DepDate: Departure date
- Visa: Reason for immigration (Business/Pleasure/Student)
- AverageTemperature: Average temperature of destination city

The fact table will be saved to parque files by city.

Note how we have trimmed down the datasets to represent just the data we re interested in, that of immigration into the US and city temperature. 

#### 3.2 Mapping Out Data Pipeline
The pipeline can be sumerised as follows:

- Clean immigration data using the clean_immigration_data() function developed in step 2. This will result in a spark data frame covering 1 month for each file passed.
- Clean temperature data using the clean_temperature_data() funcion developed in step 2. This will result in a spark data frame of temperature data.
- Create staging table staging_demographics by selecting columns from cleaned immigration data and write to parquet file partitioned by field I94Port.
- Create staging table staging_temperature by selecting columns from cleaned temperature data and write to parquet file partitioned by field I94Port.
- Create immigration_events table by joining immigration and temperature staging tables on I94Port. Write to parquet file partitioned by I94Port

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

##### staging_demographics

In [18]:
#immigration_data = '/data/18-83510-I94-Data-2016/*.sas7bdat'
immigration_data = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

# load and clean the immigration data
staging_demographics = clean_immigration_data(immigration_data)

# tidy up the column names inline with the conceptual data model
staging_demographics = staging_demographics.withColumnRenamed('i94yr', 'I94Year')\
                                           .withColumnRenamed('i94mon', 'I94Month')\
                                           .withColumnRenamed('i94cit', 'I94City')\
                                           .withColumnRenamed('i94port', 'I94Port')\
                                           .withColumnRenamed('i94mode', 'I94Mode')\
                                           .withColumnRenamed('i94bir', 'I94Birthday')\
                                           .withColumnRenamed('arrdate', 'ArrDate')\
                                           .withColumnRenamed('depdate', 'DepDate')\
                                           .withColumnRenamed('i94visa', 'I94Visa')

# now write immigration staging table to parquet files. We partition by I94Port
staging_demographics.write.mode('append').partitionBy('I94Port').parquet('/results/staging_demographics.parquet')

In [19]:
staging_demographics.columns

['I94Year',
 'I94Month',
 'I94City',
 'I94Port',
 'ArrDate',
 'I94Mode',
 'DepDate',
 'I94Birthday',
 'I94Visa']

In [20]:
staging_demographics.show(3)

+-------+--------+-------+-------+-------+-------+-------+-----------+-------+
|I94Year|I94Month|I94City|I94Port|ArrDate|I94Mode|DepDate|I94Birthday|I94Visa|
+-------+--------+-------+-------+-------+-------+-------+-----------+-------+
| 2016.0|     4.0|  692.0|    XXX|20573.0|   null|   null|       37.0|    2.0|
| 2016.0|     4.0|  254.0|    ATL|20551.0|    1.0|   null|       25.0|    3.0|
| 2016.0|     4.0|  101.0|    WAS|20545.0|    1.0|20691.0|       55.0|    2.0|
+-------+--------+-------+-------+-------+-------+-------+-----------+-------+
only showing top 3 rows



##### staging_temperature

In [21]:
# load and clean the temperature data
staging_temperature = clean_temperature_data('../../data2/GlobalLandTemperaturesByCity.csv')

# select the data and rename columns inline with the conceptual data model 
staging_temperature = staging_temperature.withColumnRenamed('i94port', 'I94Port')\
    .select(['AverageTemperature', 'City', 'Country', 'Latitude', 'Longitude', 'I94Port'])

# now write temperature dimension table to parquet files partitioned by I94Port
staging_temperature.write.mode('append').partitionBy('I94Port').parquet('/results/staging_temperature.parquet')

In [22]:
staging_temperature.columns

['AverageTemperature', 'City', 'Country', 'Latitude', 'Longitude', 'I94Port']

In [23]:
staging_temperature.show(3)

+------------------+-------+-------------+--------+---------+-------+
|AverageTemperature|   City|      Country|Latitude|Longitude|I94Port|
+------------------+-------+-------------+--------+---------+-------+
|            26.901|    Ife|      Nigeria|   7.23N|    4.05E|    888|
|            15.488|  Perth|    Australia|  31.35S|  114.97E|    PER|
|            -1.977|Seattle|United States|  47.42N|  121.97W|    SEA|
+------------------+-------+-------------+--------+---------+-------+
only showing top 3 rows



##### immigration_events

In [24]:
# Create the fact table by joining the immigration and temperature staging tables
immigration_events = staging_demographics.join(staging_temperature,  'I94Port')\
                    .select(\
                        col('I94Year').alias('Year'),
                        col('I94Month').alias('Month'),
                        col('I94City').alias('City'),
                        col('I94Port').alias('Port'),
                        col('I94Mode').alias('Mode'),
                        col('I94Birthday').alias('Birthday'),
                        'ArrDate',
                        'DepDate',
                        'AverageTemperature')

# Write fact table to parquet files partitioned by Port
immigration_events.write.mode('append').partitionBy('Port').parquet('/results/immigration_events.parquet')

In [25]:
immigration_events.columns

['Year',
 'Month',
 'City',
 'Port',
 'Mode',
 'Birthday',
 'ArrDate',
 'DepDate',
 'AverageTemperature']

In [26]:
immigration_events.show(3)

+------+-----+-----+----+----+--------+-------+-------+------------------+
|  Year|Month| City|Port|Mode|Birthday|ArrDate|DepDate|AverageTemperature|
+------+-----+-----+----+----+--------+-------+-------+------------------+
|2016.0|  4.0|111.0| SNA| 1.0|    30.0|20545.0|20547.0| 7.168999999999999|
|2016.0|  4.0|114.0| SNA| 1.0|    37.0|20545.0|20562.0| 7.168999999999999|
|2016.0|  4.0|117.0| SNA| 1.0|    51.0|20545.0|20559.0| 7.168999999999999|
+------+-----+-----+----+----+--------+-------+-------+------------------+
only showing top 3 rows



#### 4.2 Data Quality Checks
Check the size of the tables, and ensure they joined correctly.
 
Run Quality Checks

In [27]:
# simple check on the size of the table
def check_table_not_empty(table):
    return len(table.head(1)) > 0  

In [28]:
check_table_not_empty(staging_demographics)

True

In [29]:
check_table_not_empty(staging_temperature)

True

In [30]:
check_table_not_empty(immigration_events)

True

#### 4.3 Data dictionary 

##### dim_demographics

- I94Year: Year as a 4 digit number
- I94Month: Month as a numeric
- I94City: 3 digit code of origin city
- I94Port: 3 character code of destination city
- I94Mode: 1 digit travel code
- I94Birthday: Age of Respondent in Years
- ArrDate: Arrival date
- DepDate: Departure date
- I94Visa: Reason for immigration (Business/Pleasure/Student)

##### dim_temperate

- I94Port: Code of destination city (pulled from immigration data during cleanup step)
- AverageTemperature: Average temperature
- City: City name
- Country: Country name
- Latitude: Latitude
- Longitude: Longitude

##### fact_immigration

- Year: Year as a 4 digit number
- Month: Month as a numeric
- City: 3 digit code of origin city
- Destination: 3 character code of destination city
- Mode: 1 digit travel code
- Birthday: Age of Respondent in Years
- ArrDate: Arrival date
- DepDate: Departure date
- Visa: Reason for immigration (Business/Pleasure/Student)
- AverageTemperature: Average temperature of destination city

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.

To explore the datasets it is simpler via the rich API that Pandas provide. So this was chosen as the initial tool along with the sample data.

For the production side of things (building the data pipeline), Spark was chosen since it can handle large amounts of data simply by scaling up the hardware.

* Propose how often the data should be updated and why.

The immigration dataset is provided monthly, so a monthy run would be appropriate.

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 
 Increase the size of the spark cluster (and so workers) running the batch job. The spark cluster is only required monthly, so workers could be provisioned as required with autoscaling.
 
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 
 Move the ETL into an Airflow pipeline with an SLA for 7am.
 
 * The database needed to be accessed by 100+ people.
 
 In this scenario, we could publish the parquet files with read only access to HDFS.